# Battery Swapping Network Optimization
## Based on "Swap-locally, Charge-centrally" Paper

This notebook helps you:
1. Validate your current setup against the paper's framework
2. Calculate optimal battery inventory levels
3. Analyze sensitivity to different parameters
4. Plan for capacity expansion

In [ ]:
import numpy as np
from scipy.stats import norm
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from battery_analysis import SimpleBatteryModel

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

## 1. Your Current Setup - Muhanga Operation

In [ ]:
# Define your current parameters
N_VEHICLES = 200
SWAPS_PER_VEHICLE_PER_DAY = 2
CURRENT_BATTERIES = 300
CURRENT_CHARGING_PORTS = 100
CHARGING_TIME_HOURS = 3.0
TRANSPORT_TIME_HOURS = 0.5  # Average time from industrial park to stations
BATTERY_COST = 450  # USD

print(f"Fleet size: {N_VEHICLES} vehicles")
print(f"Swap frequency: {SWAPS_PER_VEHICLE_PER_DAY} times/day")
print(f"Total daily swaps: {N_VEHICLES * SWAPS_PER_VEHICLE_PER_DAY}")
print(f"Current battery inventory: {CURRENT_BATTERIES}")
print(f"Current charging capacity: {CURRENT_CHARGING_PORTS} ports")
print(f"Battery-to-vehicle ratio: {CURRENT_BATTERIES/N_VEHICLES:.2f}x")

## 2. Calculate Requirements Using Paper's Framework

### Result 1: Battery Deficit
$$E[D] = \Delta \mu$$

Where:
- $D$ = Number of batteries being charged/transported
- $\Delta$ = Effective charging time (TC + TT)
- $\mu$ = Demand rate (swaps per hour)

In [ ]:
model = SimpleBatteryModel(
    n_vehicles=N_VEHICLES,
    swaps_per_vehicle_per_day=SWAPS_PER_VEHICLE_PER_DAY,
    charging_time_hours=CHARGING_TIME_HOURS,
    transport_time_hours=TRANSPORT_TIME_HOURS,
    service_level=0.95
)

# Basic flow analysis
circ = model.calculate_batteries_in_circulation()

print("=" * 60)
print("BATTERY FLOW ANALYSIS (Result 1 from paper)")
print("=" * 60)
print(f"Demand rate (μ): {circ['demand_rate_mu']:.2f} swaps/hour")
print(f"Effective charging time (Δ): {circ['effective_time_Delta']:.2f} hours")
print(f"Expected battery deficit E[D]: {circ['batteries_charging_transport']:.1f} batteries")
print()
print("This means at any given time:")
print(f"  • {circ['batteries_in_vehicles']} batteries in vehicles (1 per vehicle)")
print(f"  • {circ['batteries_charging_transport']:.1f} batteries charging/in-transit")
print(f"  • Total minimum: {circ['batteries_in_vehicles'] + circ['batteries_charging_transport']:.1f} batteries")

## 3. Total Battery Requirements (with Safety Stock)

In [ ]:
req = model.calculate_total_batteries_needed()

print("=" * 60)
print("TOTAL BATTERY REQUIREMENTS")
print("=" * 60)
print(f"Batteries in vehicles: {req['batteries_in_vehicles']:.0f}")
print(f"Batteries in circulation: {req['batteries_in_system']:.1f}")
print(f"Safety stock (95% SL): {req['safety_stock']:.1f}")
print(f"-" * 60)
print(f"TOTAL NEEDED: {req['total_batteries']:.0f} batteries")
print(f"Battery-to-vehicle ratio: {req['battery_to_vehicle_ratio']:.2f}x")
print()
print(f"Your current setup: {CURRENT_BATTERIES} batteries")
print(f"Surplus: {CURRENT_BATTERIES - req['total_batteries']:.0f} batteries ({(CURRENT_BATTERIES/req['total_batteries']-1)*100:.1f}% above minimum)")

# Visualize breakdown
fig, ax = plt.subplots(1, 2, figsize=(14, 5))

# Battery breakdown
categories = ['In Vehicles', 'In Circulation', 'Safety Stock', 'Your Surplus']
values = [
    req['batteries_in_vehicles'],
    req['batteries_in_system'],
    req['safety_stock'],
    CURRENT_BATTERIES - req['total_batteries']
]
colors = ['#2ecc71', '#3498db', '#f39c12', '#95a5a6']

ax[0].bar(categories, values, color=colors)
ax[0].set_ylabel('Number of Batteries')
ax[0].set_title('Battery Inventory Breakdown')
ax[0].axhline(y=CURRENT_BATTERIES, color='r', linestyle='--', label='Your Total (300)')
ax[0].legend()
ax[0].grid(axis='y', alpha=0.3)

# Pie chart
ax[1].pie(values, labels=categories, colors=colors, autopct='%1.1f%%', startangle=90)
ax[1].set_title('Battery Allocation (Your 300 Batteries)')

plt.tight_layout()
plt.show()

## 4. Charging Capacity Analysis

In [ ]:
charging = model.calculate_charging_capacity()

print("=" * 60)
print("CHARGING CAPACITY ANALYSIS")
print("=" * 60)
print(f"Batteries processed per hour: {charging['batteries_per_hour']:.2f}")
print(f"Theoretical minimum ports: {charging['theoretical_min']:.1f}")
print(f"Recommended ports (10% margin): {charging['recommended']:.1f}")
print(f"Your current capacity: {CURRENT_CHARGING_PORTS} ports")
print(f"Utilization rate: {(charging['theoretical_min']/CURRENT_CHARGING_PORTS*100):.1f}%")
print()
print(f"✓ You have {CURRENT_CHARGING_PORTS - charging['recommended']:.0f} extra ports of capacity")
print(f"✓ This provides buffer for peak demand and maintenance")

## 5. Sensitivity Analysis: Key Parameters

### 5.1 Impact of Swap Frequency

In [ ]:
swap_frequencies = np.arange(1.0, 5.1, 0.25)
batteries_needed = []
charging_ports_needed = []

for swaps in swap_frequencies:
    m = SimpleBatteryModel(
        n_vehicles=N_VEHICLES,
        swaps_per_vehicle_per_day=swaps,
        charging_time_hours=CHARGING_TIME_HOURS,
        transport_time_hours=TRANSPORT_TIME_HOURS,
        service_level=0.95
    )
    req = m.calculate_total_batteries_needed()
    chg = m.calculate_charging_capacity()
    batteries_needed.append(req['total_batteries'])
    charging_ports_needed.append(chg['recommended'])

fig, ax = plt.subplots(1, 2, figsize=(14, 5))

# Batteries vs swap frequency
ax[0].plot(swap_frequencies, batteries_needed, 'b-', linewidth=2, label='Required batteries')
ax[0].axhline(y=CURRENT_BATTERIES, color='r', linestyle='--', linewidth=2, label='Your current (300)')
ax[0].axvline(x=SWAPS_PER_VEHICLE_PER_DAY, color='g', linestyle=':', alpha=0.7, label='Your current (2/day)')
ax[0].set_xlabel('Swaps per Vehicle per Day')
ax[0].set_ylabel('Total Batteries Required')
ax[0].set_title('Battery Requirements vs Swap Frequency')
ax[0].legend()
ax[0].grid(alpha=0.3)

# Charging ports vs swap frequency
ax[1].plot(swap_frequencies, charging_ports_needed, 'b-', linewidth=2, label='Required ports')
ax[1].axhline(y=CURRENT_CHARGING_PORTS, color='r', linestyle='--', linewidth=2, label='Your current (100)')
ax[1].axvline(x=SWAPS_PER_VEHICLE_PER_DAY, color='g', linestyle=':', alpha=0.7, label='Your current (2/day)')
ax[1].set_xlabel('Swaps per Vehicle per Day')
ax[1].set_ylabel('Charging Ports Required')
ax[1].set_title('Charging Capacity vs Swap Frequency')
ax[1].legend()
ax[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

# Find maximum swap frequency you can support
max_swaps_batteries = swap_frequencies[np.where(np.array(batteries_needed) <= CURRENT_BATTERIES)[0][-1]]
max_swaps_charging = swap_frequencies[np.where(np.array(charging_ports_needed) <= CURRENT_CHARGING_PORTS)[0][-1]]

print(f"With your current setup:")
print(f"  • Battery inventory can support up to {max_swaps_batteries:.2f} swaps/vehicle/day")
print(f"  • Charging capacity can support up to {max_swaps_charging:.2f} swaps/vehicle/day")
print(f"  • Overall capacity: {min(max_swaps_batteries, max_swaps_charging):.2f} swaps/vehicle/day")

### 5.2 Impact of Charging Time

In [ ]:
charging_times = np.arange(1.5, 5.1, 0.25)
batteries_needed = []

for tc in charging_times:
    m = SimpleBatteryModel(
        n_vehicles=N_VEHICLES,
        swaps_per_vehicle_per_day=SWAPS_PER_VEHICLE_PER_DAY,
        charging_time_hours=tc,
        transport_time_hours=TRANSPORT_TIME_HOURS,
        service_level=0.95
    )
    req = m.calculate_total_batteries_needed()
    batteries_needed.append(req['total_batteries'])

plt.figure(figsize=(10, 6))
plt.plot(charging_times, batteries_needed, 'b-', linewidth=2)
plt.axhline(y=CURRENT_BATTERIES, color='r', linestyle='--', linewidth=2, label='Your current batteries (300)')
plt.axvline(x=CHARGING_TIME_HOURS, color='g', linestyle=':', alpha=0.7, label='Your current charging time (3h)')
plt.xlabel('Charging Time (hours)')
plt.ylabel('Total Batteries Required')
plt.title('Impact of Charging Time on Battery Requirements')
plt.legend()
plt.grid(alpha=0.3)
plt.show()

print("Key insights:")
print(f"  • Every 1 hour increase in charging time requires ~{(batteries_needed[-1]-batteries_needed[0])/(charging_times[-1]-charging_times[0]):.0f} more batteries")
print(f"  • Faster charging (2h) would need only ~{batteries_needed[2]:.0f} batteries")
print(f"  • Slower charging (4h) would need ~{batteries_needed[-4]:.0f} batteries")

### 5.3 Impact of Service Level

In [ ]:
service_levels = np.arange(0.80, 1.00, 0.01)
batteries_needed = []

for sl in service_levels:
    m = SimpleBatteryModel(
        n_vehicles=N_VEHICLES,
        swaps_per_vehicle_per_day=SWAPS_PER_VEHICLE_PER_DAY,
        charging_time_hours=CHARGING_TIME_HOURS,
        transport_time_hours=TRANSPORT_TIME_HOURS,
        service_level=sl
    )
    req = m.calculate_total_batteries_needed()
    batteries_needed.append(req['total_batteries'])

plt.figure(figsize=(10, 6))
plt.plot(service_levels * 100, batteries_needed, 'b-', linewidth=2)
plt.axhline(y=CURRENT_BATTERIES, color='r', linestyle='--', linewidth=2, label='Your current batteries (300)')
plt.axvline(x=95, color='g', linestyle=':', alpha=0.7, label='95% service level')
plt.xlabel('Service Level (%)')
plt.ylabel('Total Batteries Required')
plt.title('Trade-off: Service Level vs Battery Investment')
plt.legend()
plt.grid(alpha=0.3)
plt.show()

# Calculate your actual service level
your_sl_idx = np.where(np.array(batteries_needed) <= CURRENT_BATTERIES)[0][-1]
your_achievable_sl = service_levels[your_sl_idx]

print(f"With 300 batteries, you can achieve:")
print(f"  • Service level: {your_achievable_sl*100:.1f}%")
print(f"  • This means batteries available {your_achievable_sl*100:.1f}% of the time")
print(f"  • Stockout probability: {(1-your_achievable_sl)*100:.1f}%")

## 6. Expansion Planning

### What if you expand to 400 or 500 vehicles?

In [ ]:
fleet_sizes = [200, 300, 400, 500]
results = []

for fleet in fleet_sizes:
    m = SimpleBatteryModel(
        n_vehicles=fleet,
        swaps_per_vehicle_per_day=SWAPS_PER_VEHICLE_PER_DAY,
        charging_time_hours=CHARGING_TIME_HOURS,
        transport_time_hours=TRANSPORT_TIME_HOURS,
        service_level=0.95
    )
    req = m.calculate_total_batteries_needed()
    chg = m.calculate_charging_capacity()
    
    results.append({
        'Fleet Size': fleet,
        'Batteries Needed': int(req['total_batteries']),
        'Battery Ratio': f"{req['battery_to_vehicle_ratio']:.2f}x",
        'Charging Ports': int(chg['recommended']),
        'Battery Investment': f"${int(req['total_batteries'] * BATTERY_COST):,}",
        'Additional Batteries': int(req['total_batteries']) - CURRENT_BATTERIES if fleet > 200 else 0,
        'Additional Investment': f"${int((req['total_batteries'] - CURRENT_BATTERIES) * BATTERY_COST):,}" if fleet > 200 else "$0"
    })

df = pd.DataFrame(results)
print("\nEXPANSION PLANNING SCENARIOS")
print("=" * 80)
print(df.to_string(index=False))
print("\nKey takeaways:")
print(f"  • Battery ratio stays relatively constant (~1.37x)")
print(f"  • Doubling fleet to 400 vehicles requires ~{results[2]['Batteries Needed']} batteries")
print(f"  • This is {results[2]['Additional Batteries']} additional batteries = {results[2]['Additional Investment']}")

## 7. Cost Optimization

### Trade-off between battery investment and service level

In [ ]:
# Calculate cost of stockouts vs battery investment
service_levels = np.arange(0.80, 0.99, 0.01)
battery_costs = []
stockout_costs = []

LOST_REVENUE_PER_STOCKOUT = 5  # USD per missed swap (estimate)
SWAPS_PER_YEAR = N_VEHICLES * SWAPS_PER_VEHICLE_PER_DAY * 365

for sl in service_levels:
    m = SimpleBatteryModel(
        n_vehicles=N_VEHICLES,
        swaps_per_vehicle_per_day=SWAPS_PER_VEHICLE_PER_DAY,
        charging_time_hours=CHARGING_TIME_HOURS,
        transport_time_hours=TRANSPORT_TIME_HOURS,
        service_level=sl
    )
    req = m.calculate_total_batteries_needed()
    
    # Battery investment cost
    battery_investment = req['total_batteries'] * BATTERY_COST
    battery_costs.append(battery_investment)
    
    # Annual stockout cost
    stockout_probability = 1 - sl
    expected_stockouts_per_year = SWAPS_PER_YEAR * stockout_probability
    annual_stockout_cost = expected_stockouts_per_year * LOST_REVENUE_PER_STOCKOUT
    stockout_costs.append(annual_stockout_cost)

# Total cost over 3 years
total_costs = np.array(battery_costs) + np.array(stockout_costs) * 3

plt.figure(figsize=(12, 6))
plt.plot(service_levels * 100, battery_costs, label='Battery Investment', linewidth=2)
plt.plot(service_levels * 100, np.array(stockout_costs) * 3, label='3-Year Stockout Cost', linewidth=2)
plt.plot(service_levels * 100, total_costs, label='Total Cost', linewidth=2, linestyle='--', color='red')
plt.axvline(x=95, color='g', linestyle=':', alpha=0.7, label='95% service level')
plt.xlabel('Service Level (%)')
plt.ylabel('Cost (USD)')
plt.title('Cost Optimization: Battery Investment vs Stockout Costs')
plt.legend()
plt.grid(alpha=0.3)
plt.show()

# Find optimal service level
optimal_idx = np.argmin(total_costs)
optimal_sl = service_levels[optimal_idx]

print(f"\nCOST OPTIMIZATION RESULTS")
print(f"=" * 60)
print(f"Optimal service level: {optimal_sl*100:.1f}%")
print(f"Total cost minimized at: ${total_costs[optimal_idx]:,.0f}")
print(f"\nNote: This assumes ${LOST_REVENUE_PER_STOCKOUT} lost revenue per stockout")
print(f"Adjust LOST_REVENUE_PER_STOCKOUT based on your actual business impact")

## 8. Summary & Recommendations

In [ ]:
print("=" * 70)
print("SUMMARY & RECOMMENDATIONS")
print("=" * 70)
print()
print("✓ CURRENT SETUP VALIDATION:")
print(f"  • Your 1.5x ratio (300 batteries) is well-aligned with the paper")
print(f"  • Provides ~26 battery surplus above theoretical minimum")
print(f"  • Charging capacity (100 ports) is sufficient with 50% utilization")
print()
print("✓ CAPACITY HEADROOM:")
print(f"  • Can support up to ~{max_swaps_batteries:.1f} swaps/vehicle/day (battery limited)")
print(f"  • Can support up to ~{max_swaps_charging:.1f} swaps/vehicle/day (charging limited)")
print(f"  • Currently at {SWAPS_PER_VEHICLE_PER_DAY}/day, so {(max_swaps_batteries/SWAPS_PER_VEHICLE_PER_DAY - 1)*100:.0f}% growth capacity")
print()
print("⚠ CRITICAL MONITORING POINTS:")
print(f"  • Track actual swap frequency - if it increases to 3/day, add batteries")
print(f"  • Monitor transport times - longer trips require more inventory")
print(f"  • Measure actual stockout rates to validate service level")
print()
print("💡 OPTIMIZATION OPPORTUNITIES:")
print(f"  • If you can reduce charging time to 2.5h, you'd need only ~270 batteries")
print(f"  • Faster transport (20min vs 30min) saves ~5 batteries")
print(f"  • Consider dynamic pricing during peak hours to smooth demand")
print()
print("📊 NEXT STEPS:")
print(f"  1. Collect real operational data (actual swap times, demand patterns)")
print(f"  2. Measure stockout frequency to validate service level")
print(f"  3. Use this model to plan expansion (e.g., adding new stations)")
print(f"  4. Optimize transport routes to minimize TT")
print("=" * 70)